# В данном ноутбуке описан код построения пайплайна с использованием гибридного поиска (векторный + полнотекстовый) и созданием цепочки для ответов на вопросы на основе контекста.

Для решения использовано:
- Эмбеддинг модель [ruElectra-large](https://huggingface.co/ai-forever/ruElectra-large)
- ChromaDB как векторная база данных
- BM25 для поиска по тексту
- Gpt 3.5 для ответов

In [ ]:
%pip install -r requirements.txt

## Для теста русскоязыной QA возьмём пример из датасета sberquad

In [43]:
import pandas as pd

df = pd.read_parquet('sberquad/sberquad/train-00000-of-00001.parquet')

df.head()

,id,title,context,question,answers
0,62310,SberChallenge,В протерозойских отложениях органические остат...,чем представлены органические остатки?,{'text': ['известковыми выделениями сине-зелён...
1,28101,SberChallenge,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,"{'text': ['нитевидные водоросли, грибные нити'..."
2,48834,SberChallenge,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,"{'text': ['органические остатки'], 'answer_sta..."
3,83056,SberChallenge,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,{'text': ['скопления графито-углистого веществ...
4,5816,SberChallenge,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,{'text': ['в результате разложения Corycium en...


In [44]:
context = df['context'][0]
question = df['question'][0]

In [45]:
context

'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.'

In [46]:
question

'чем представлены органические остатки?'

## Пайплайн построения QA системы

In [47]:
from langchain_core.documents import Document
doc = Document(page_content=context, metadata={"source": "local"})
documents = [doc]

### Разбиваем контекст на чанки

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                          chunk_overlap=100)
chunks = splitter.split_documents(documents)

In [49]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
langchain_embeddings = SentenceTransformerEmbeddings(model_name="embedding_models/ruElectra-large")

No sentence-transformers model found with name embedding_models/ruElectra-large. Creating a new one with mean pooling.


### Векторизуем каждый чанк из документов и сохраняем в ChromaDB

In [50]:
from langchain.vectorstores import Chroma
chroma_persist_directory = "chroma_db"

vector_store = Chroma(
    persist_directory=chroma_persist_directory,
    embedding_function=langchain_embeddings
)
vector_store.add_documents(chunks)
vector_store.persist()

### Устанавливаем топ сколько документов будет выдавать ретриверы

In [51]:
NUM_DOCS = 3

### Векторный ретривер

In [ ]:
vectorstore_retreiver = vector_store.as_retriever(search_kwargs={"k": NUM_DOCS})

In [ ]:
vectorstore_retreiver.invoke(input=question)

### Текстовый ретривер

In [53]:
from langchain_community.retrievers import BM25Retriever

keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k = NUM_DOCS

In [54]:
keyword_retriever.invoke(input=question)

[Document(metadata={'source': 'local'}, page_content='В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками'),
 Document(metadata={'source': 'local'}, page_content='Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся'),
 Document(metadata={'source': 'local'}, page_content='близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.')]

### Ансамблирование ретриверов для создания гибридного поиска по документам

In [55]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.2, 0.8])

### Объявление LLM для ответов на основе выданного поиском документов

In [56]:
from langchain_community.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

### Скачивание промпта для ответа на вопросы на основе контекста

In [57]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

c:\Users\vladimir\Visual_Studio_Code_projects\Sirius\.venv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### Создание цепочки из ретривера и LLM

In [58]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=ensemble_retriever, chain_type_kwargs={"prompt": prompt}
)

In [59]:
result = qa_chain({"query": question})
result["result"]

'Органические остатки представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Также встречаются скопления графито-углистого вещества и железистые продукты жизнедеятельности бактерий.'